In [1]:
import yfinance as yf

oro = yf.Ticker("GLD")

In [2]:
oro_data = oro.history(period="1y", interval="1d")
oro_data


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2024-10-04 00:00:00-04:00,245.000000,246.690002,244.050003,245.000000,5947800,0.0,0.0,0.0
2024-10-07 00:00:00-04:00,244.580002,244.820007,243.809998,244.169998,3849000,0.0,0.0,0.0
2024-10-08 00:00:00-04:00,243.789993,244.039993,240.630005,242.369995,7669800,0.0,0.0,0.0
2024-10-09 00:00:00-04:00,241.160004,241.839996,240.639999,241.050003,4120500,0.0,0.0,0.0
2024-10-10 00:00:00-04:00,241.770004,243.020004,241.509995,242.820007,4792700,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-09-29 00:00:00-04:00,351.790009,352.829987,351.279999,352.459991,15719600,0.0,0.0,0.0
2025-09-30 00:00:00-04:00,351.130005,355.570007,350.869995,355.470001,13312400,0.0,0.0,0.0
2025-10-01 00:00:00-04:00,356.170013,356.970001,354.790009,356.029999,15573600,0.0,0.0,0.0
